# Import Librairie:

In [1]:
from PIL import Image
import pytesseract
from pdf2image import convert_from_path
from sqlalchemy import create_engine
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline
import cv2
from pytesseract import Output
import pandas as pd
import tabula
import pdfplumber

# Contexte:

Votre client souhaiterait extraire les tableaux de données des CRA (Compte-Rendu d'Activite).   
Ce type de document est complexe, il s'agit souvent de document format PDF de nombreuses pages, avec du texte mais également éléments des tableaux de chiffre qu'il serait intéressant de comparer pour faire des prédictions et évaluer les métriques par rapport aux attentes.  

​

Pour ce faire vous devrez utiliser les techniques d'OCR pour récupérer les données des fichiers PDF, puis de stocker ces données tabulaires dans un système de gestionnaire de bases de données relationnelles (par exemple MySQL).  

Le client souhaite que vous travaillez de manière itérative :  

dans un premier temps faites une veille pour tester la faisabalité des extractions de tableaux de chiffres d'un fichier PDF (le fichier data pourra vous être utile)  
puis l'extraction de différents tableaux dans une même page d'un PDF (le fichier test1 pourra être une bonne base).  
​

Bien sûr, pensez à automatiser au maximum votre code.  

# Tabula:

Pour ce pdf tabula a facilement récupéré le tableau même s'ils sont ouvert sur les cotés

In [2]:
def convert_pdf_to_dataframe(pdf,position):
    # pdf mean the path to the location of your file to convert
    df = tabula.read_pdf(pdf, pages='all', stream= True, multiple_tables=True)
    # number represente the place of a dataset in order in your pdf
    if pdf == "data.pdf" and position == 0:
        df = df[0]
        df = df.rename(columns={"Unnamed: 0": "model"})
        return df
        
    else:
        return df[position]

In [144]:
df_car = convert_pdf_to_dataframe("data.pdf",0)
df_car

IndexError: list index out of range

# PDF Plumber:

ppour le test1 Plumber a mieux récupéré les differents tableau que tabula

In [140]:
def extract_table(pdf_file,position,transpose,header):
        # pdf_file argument mean the path to the location of your file to convert
    with pdfplumber.open(pdf_file) as pdf:
        first_page = pdf.pages[0].find_tables()
        # position argument represente the place of the dataset in order in your pdf
        df = first_page[position].extract()
        df = pd.DataFrame(df).replace({'\n': ''}, regex=True)
        # loop for place or not the first row as a header for the column name of dataset
        if transpose == True:
            df = df.transpose()
        else:
            None
        if header == True :
            new_header = df.iloc[0] #grab the first row for the header
            df = df[1:] #take the data less the header row
            df.columns = new_header #set the header row as the df header
        else:
            None       
        
    return df

In [143]:
df = extract_table("test1.pdf",4,True,True)
df

,Topic,Has inspection frequency target been achieved for the batch(es) concerned?,"If inspection target has not been achieved, indicate what the inspection gaps (or missing inspections) are.","If inspection target has not been achieved, provide justification why it is acceptable for the batch(es) to be released.",No. of Observation Sheets raised,Have all observation sheets been satisfactorily responded to and closed at time of issuing the clearance note?,Should a final Inspection be targeted before the QRC is accepted? (proposal of DI or EIRA to be endorsed by HPC)
1,Response,Y,N,Y,,,N
2,Comments,Except for activities that haven't occured yet,,,N/A,N/A,
3,Notes,Note: This relates to the completeness of the ...,"For example, provide list of the guides that h...",For example: Provide argues that the guides no...,N/A,1) Provide list of Open Observation Sheets.2) ...,"If targeted, a flash report will be required w..."
4,Responsibility,Activity Pilot/Region Leader,Activity Pilot/Region Leader,Activity Pilot/Region Leader,Activity Pilot/Region Leader,Activity Pilot/Region Leader,Equipment Engineer/Inspection Pilot


# Keras_ocr

Api utilisant l'ocr de keras avec des model déjà entrainé.

In [1]:
import keras_ocr
# pipeline contenant le modèle pré entrainer 
pipeline = keras_ocr.pipeline.Pipeline()

Looking for C:\Users\mattb\.keras-ocr\craft_mlt_25k.h5
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Looking for C:\Users\mattb\.keras-ocr\crnn_kurapan.h5


In [ ]:
from pdf2image import convert_from_path
pages = convert_from_path("test2.pdf", 500)
for page in pages:
    page.save('test2.jpg', 'JPEG')

In [178]:
images = keras_ocr.tools.read('test2.jpg')

In [179]:
type([img])

list

In [181]:
# Each list of predictions in prediction_groups is a list of
# (word, box) tuples.

prediction_groups = pipeline.recognize([images])

Les informations ont été parfaitement extraite du document.

In [ ]:
# Plot the predictions
fig, axs = plt.subplots(nrows=len(images), figsize=(20, 20))
for ax, image, predictions in zip(axs, images, prediction_groups):
    keras_ocr.tools.drawAnnotations(image=image, predictions=predictions, ax=ax)